# Trabalho 3

Grupo 5  
Filipe Barbosa a77252  
Hugo Ferreira a78555

In [1]:
from z3 import *
import math

### Problema 2

Pretende-se construir um  autómato híbrido que modele uma situação definida por 3 navios a navegar num lago infinito. Cada navio é caracterizado pela sua posição no plano $\,(x,y)\,$, a sua rota medida num ângulo com o eixo horizontal em unidades de $15^o$, e uma velocidade que assume apenas 2 valores:  $1\, \texttt{m/s}\,$ (“low”) e $\,10\,\mathtt{m/s}\,$ (“hight”).

Criamos a matrix ang que contem os senos e cossenos predefinidos.
Se seguida definimos o coeficiente de proximidade $r = 1$

In [2]:
# Matriz com os angulos predifinidos
sen = []
cos = []
ang = []
for i in range(24):
    sen.append(math.sin(math.radians(15*i)))
    cos.append(math.cos(math.radians(15*i)))

ang.append(sen)
ang.append(cos)

# coeficiente de proximidade
r = 1

A função "declare" cria a $i$-ésima cópia das variáveis de estado, agrupadas num dicionário que nos permite aceder às mesmas pelo nome.
Declaramos de seguida os vários modos, $INIT$, $HIGH$ e $LOW$.

In [3]:
#modos init, e das velocidades
Mode, (INIT,HIGH,LOW) = EnumSort('Mode', ('INIT','HIGH','LOW'))

#declara a i-esima copia
def declare(i,x):
    s = {}
    s['t'] = Real('t'+str(i)+str(x))
    s['m'] = Const('m'+str(i)+str(x),Mode)
    s['x'] = Real('x'+str(i)+str(x))
    s['y'] = Real('y'+str(i)+str(x))
    s['v'] = Int('v'+str(i)+str(x))
    s['rota'] = Int('rota'+str(i)+str(x))
    return s

Define-se então a função $init$ que declara o estado inicial do programa.
O tempo começa no 0, o modo inicial é o $INIT$ a velocidade e rota encontram-se a 0.

$$
\mathit{t} = 0 \wedge \mathit{m} = INIT \wedge \mathit{v} = 0 \wedge \mathit{rota} = 0
$$

In [4]:
#declara estado inicial
def init(s):
    return And(s['t'] == 0, s['m'] == INIT, s['v'] == 0, s['rota'] == 0)

As transições untimed são caracterizadas pelo seguinte predicado:

$$
\begin{array}{c}
( \mathit{m} = INIT \wedge \mathit{m}' = HIGH \wedge \mathit{x}' = \mathit{x} \wedge \mathit{y}' = \mathit{y} \wedge  \mathit{t}' = \mathit{t} \wedge \mathit{v}' = 10 \wedge rota' = rota)\\
\vee\\
(\mathit{m} = HIGH \wedge \mathit{m}' = LOW \wedge \mathit{x}' = \mathit{x} \wedge \mathit{y}' = \mathit{y} \wedge \mathit{t}' = \mathit{t} \wedge \mathit{v} = 10 \wedge \mathit{v}' = 1 \wedge ((rota' = rota + 1 \wedge rota' \ge 0) \vee (rota' = rota - 1 \wedge rota' > 0)) \wedge \mathit{x}' \le \mathit{x} + r \wedge \mathit{x} \le \mathit{x}' + r \wedge \mathit{y}' \le \mathit{y} + r \wedge \mathit{y} \le \mathit{y}' + r \wedge \mathit{t}' \le \mathit{t} + \frac{r}{v} \wedge \mathit{t} \le \mathit{t}' + \frac{r}{v})\\
\vee\\
(\mathit{m} = LOW \wedge \mathit{m}' = HIGH \wedge \mathit{x}' = \mathit{x} \wedge \mathit{y}' = \mathit{y} \wedge \mathit{t}' = \mathit{t} \wedge \mathit{v} = 10 \wedge \mathit{v}' = 1 \wedge rota' = rota \wedge \mathit{x}' > \mathit{x} + r \wedge \mathit{x} > \mathit{x}' + r \wedge \mathit{y}' > \mathit{y} + r \wedge \mathit{y} > \mathit{y}' + r \wedge \mathit{t}' > \mathit{t} + \frac{r}{v} \wedge \mathit{t} > \mathit{t}' + \frac{r}{v})\\
\end{array}
$$

As transições timed são caracterizadas pelo seguinte predicado:

$$
\begin{array}{c}
(\mathit{m} = LOW \wedge \mathit{m}' = LOW \wedge \mathit{x}' = \mathit{x} + 1*(\mathit{t}'-\mathit{t})*cos \wedge \mathit{y}' = \mathit{y} + 1*(\mathit{t}'-\mathit{t})*sen \wedge \mathit{t}' > \mathit{t} \wedge \wedge \mathit{v} = 1 \wedge \mathit{v}' = \mathit{v} \wedge rota' = rota)\\
\vee\\
(\mathit{m} = HIGH \wedge \mathit{m}' = HIGH \wedge \mathit{x}' = \mathit{x} + 10*(\mathit{t}'-\mathit{t})*cos \wedge \mathit{y}' = \mathit{y} + 10*(\mathit{t}'-\mathit{t})*sen \wedge \mathit{t}' > \mathit{t} \wedge \mathit{v} = 10 \wedge \mathit{v}' = \mathit{v} \wedge rota' = rota)\\
\end{array}
$$

In [5]:
#funcao com as transicoes
def trans(curr,prox,sen,cos,x,y,t,v):
    # untimed
    t1_ih = And(curr['m'] == INIT, prox['m'] == HIGH, prox['x'] == curr['x'], prox['y'] == curr['y'], prox['t'] == curr['t'],                  prox['v'] == 10, prox['rota'] == curr['rota'])
    t2_hl = And(curr['m'] == HIGH, prox['m'] == LOW,  prox['x'] == curr['x'], prox['y'] == curr['y'], prox['t'] == curr['t'], curr['v'] == 10, prox['v'] == 1,  Or(And(prox['rota'] == curr['rota'] + 1, curr['rota'] >= 0), And(prox['rota'] == curr['rota'] - 1, curr['rota'] > 0)), prox['x'] <= x + r, x <= prox['x'] + r, prox['y'] <= y + r, y <= prox['y'] + r, prox['t'] <= t + r / v, t <= prox['t'] + r / v)
    t3_lh = And(curr['m'] == LOW,  prox['m'] == HIGH, prox['x'] == curr['x'], prox['y'] == curr['y'], prox['t'] == curr['t'], curr['v'] == 1,  prox['v'] == 10, prox['rota'] == curr['rota'],                        prox['x'] > x + r, x > prox['x'] + r, prox['y'] > y + r, y > prox['y'] + r, prox['t'] > r / v + t, t > prox['t'] + r / v)
    # timed
    t4_ll = And(curr['m'] == LOW,  prox['m'] == curr['m'], prox['t'] > curr['t'], curr['v'] == 1,  prox['v'] == curr['v'], prox['x'] == curr['x'] + 1 * (prox['t'] - curr['t']) * cos,  prox['y'] == curr['y'] + 1 * (prox['t'] - curr['t']) * sen,  prox['rota'] == curr['rota'])
    t5_hh = And(curr['m'] == HIGH, prox['m'] == curr['m'], prox['t'] > curr['t'], curr['v'] == 10, prox['v'] == curr['v'], prox['x'] == curr['x'] + 10 * (prox['t'] - curr['t']) * cos, prox['y'] == curr['y'] + 10 * (prox['t'] - curr['t']) * sen, prox['rota'] == curr['rota'])
    return Or(t1_ih,t2_hl,t3_lh,t4_ll,t5_hh)

Criamos agora a função de ordem superior $gera\_traco$ que, para três estados (um para cada navio), gera uma cópia das variáveis do estado, um predicado que define o estado inicial, outro que adiciona pares de transições entre pares de estados e um número n que define o tamanho do traço.

In [6]:
#funcao que gera o traco
def gera_traco(declare,init,trans,k):
    s = Solver()  
    state1 = [declare(i,1) for i in range(k)]
    state2 = [declare(i,2) for i in range(k)]
    state3 = [declare(i,3) for i in range(k)]
    s.add(init(state1[0]))
    s.add(init(state2[0]))
    s.add(init(state3[0]))
    for i in range(k-1):
        s.add(trans(state1[i],state1[i+1],ang[0][i],ang[1][i],state2[i]['x'],state2[i]['y'],state2[i]['t'],(state1[i]['v'] + state2[i]['v'])/2))
        s.add(trans(state2[i],state2[i+1],ang[0][i],ang[1][i],state3[i]['x'],state3[i]['y'],state3[i]['t'],(state2[i]['v'] + state3[i]['v'])/2))
        s.add(trans(state3[i],state3[i+1],ang[0][i],ang[1][i],state1[i]['x'],state1[i]['y'],state1[i]['t'],(state3[i]['v'] + state1[i]['v'])/2))
    if s.check() == sat:
        m = s.model()
        for i in range(k):
            print("Transicao",i+1)
            print("Navio1")
            for x in state1[i]:
                if state1[i][x].sort() != RealSort():
                    print(x,'=',m[state1[i][x]])
                else:
                    print(x,'=',float(m[state1[i][x]].numerator_as_long())/float(m[state1[i][x]].denominator_as_long()))
            print("Navio2")
            for x in state2[i]:
                if state2[i][x].sort() != RealSort():
                    print(x,'=',m[state2[i][x]])
                else:
                    print(x,'=',float(m[state2[i][x]].numerator_as_long())/float(m[state2[i][x]].denominator_as_long()))
            print("Navio3")
            for x in state3[i]:
                if state3[i][x].sort() != RealSort():
                    print(x,'=',m[state3[i][x]])
                else:
                    print(x,'=',float(m[state3[i][x]].numerator_as_long())/float(m[state3[i][x]].denominator_as_long()))
            print("\n")
                    
gera_traco(declare,init,trans,10)

Transicao 1
Navio1
t = 0.0
m = INIT
x = -2.3147842970973125
y = -2.3781146681832848
v = 0
rota = 0
Navio2
t = 0.0
m = INIT
x = -3.2206984454883774
y = -3.605258007165
v = 0
rota = 0
Navio3
t = 0.0
m = INIT
x = -1.3147842970973125
y = -1.378114668183285
v = 0
rota = 0


Transicao 2
Navio1
t = 0.0
m = HIGH
x = -2.3147842970973125
y = -2.3781146681832848
v = 10
rota = 0
Navio2
t = 0.0
m = HIGH
x = -3.2206984454883774
y = -3.605258007165
v = 10
rota = 0
Navio3
t = 0.0
m = HIGH
x = -1.3147842970973125
y = -1.378114668183285
v = 10
rota = 0


Transicao 3
Navio1
t = 0.16710772781411617
m = HIGH
x = -0.7006475964159236
y = -1.94560804276227
v = 10
rota = 0
Navio2
t = 0.07292809448516281
m = HIGH
x = -2.5162671462356956
y = -3.416506209407037
v = 10
rota = 0
Navio3
t = 0.0
m = LOW
x = -1.3147842970973125
y = -1.378114668183285
v = 1
rota = 1


Transicao 4
Navio1
t = 0.24003582229927894
m = HIGH
x = -0.06907177167849542
y = -1.5809675703364563
v = 10
rota = 0
Navio2
t = 0.24003582229927894
m = H

Agora para testar se não há colisões de navios em $k$ iterações criamos o $bmc\_always$. Adicionamos a propriedade e verificamos se ela é válida.

In [9]:
#funcao para testar invariantes
def bmc_always(declare,init,trans,inv,k):
    s = Solver()
    state1 = [declare(i,1) for i in range(k)]
    state2 = [declare(i,2) for i in range(k)]
    state3 = [declare(i,3) for i in range(k)]
    s.add(init(state1[0]))
    s.add(init(state2[0]))
    s.add(init(state3[0]))
    for i in range(k-1):
        s.add(trans(state1[i],state1[i+1],sen[i],cos[i],state2[i]['x'],state2[i]['y'],state2[i]['t'],(state1[i]['v'] + state2[i]['v'])/2))
        s.add(trans(state2[i],state2[i+1],sen[i],cos[i],state3[i]['x'],state3[i]['y'],state3[i]['t'],(state2[i]['v'] + state3[i]['v'])/2))
        s.add(trans(state3[i],state3[i+1],sen[i],cos[i],state1[i]['x'],state1[i]['y'],state1[i]['t'],(state3[i]['v'] + state1[i]['v'])/2))    
    s.add(And([Not(inv(state1[i],state2[i])) for i in range(k)]))
    s.add(And([Not(inv(state2[i],state3[i])) for i in range(k)]))
    s.add(And([Not(inv(state3[i],state1[i])) for i in range(k)]))
    if s.check() == sat:
        m = s.model()
        for i in range(k):
            print("Transicao",i)
            print("Navio1")
            for x in state1[i]:
                if state1[i][x].sort() != RealSort():
                    print(x,'=',m[state1[i][x]])
                else:
                    print(x,'=',float(m[state1[i][x]].numerator_as_long())/float(m[state1[i][x]].denominator_as_long()))
            print("Navio2")
            for x in state2[i]:
                if state2[i][x].sort() != RealSort():
                    print(x,'=',m[state2[i][x]])
                else:
                    print(x,'=',float(m[state2[i][x]].numerator_as_long())/float(m[state2[i][x]].denominator_as_long()))
            print("Navio3")
            for x in state3[i]:
                if state3[i][x].sort() != RealSort():
                    print(x,'=',m[state3[i][x]])
                else:
                    print(x,'=',float(m[state3[i][x]].numerator_as_long())/float(m[state3[i][x]].denominator_as_long()))
            print("\n")
        print ("Nao há colisões para "+str(k)+" estados") 
        return
        
    print ("Há colisões para "+str(k)+" estados") 

Para não haver colisões entre navios as coordenadas x e y não podem ser iguais entre dois navios no mesmo tempo.

In [10]:
#invariante
def colide(s1,s2):
    return Or(And(s1['x']==s2['x'],s1['t']==s2['t']),And(s1['y']==s2['y'],s1['t']==s2['t']))

bmc_always(declare,init,trans,colide,7)

Transicao 0
Navio1
t = 0.0
m = INIT
x = -0.8155575109965235
y = -0.4901674386892491
v = 0
rota = 0
Navio2
t = 0.0
m = INIT
x = 0.15100509197431786
y = -0.4901674386892491
v = 0
rota = 0
Navio3
t = 0.0
m = INIT
x = -0.8489949080256821
y = -1.4901674386892492
v = 0
rota = 0


Transicao 1
Navio1
t = 0.0
m = HIGH
x = -0.8155575109965235
y = -0.4901674386892491
v = 10
rota = 0
Navio2
t = 0.0
m = HIGH
x = 0.15100509197431786
y = -0.4901674386892491
v = 10
rota = 0
Navio3
t = 0.0
m = HIGH
x = -0.8489949080256821
y = -1.4901674386892492
v = 10
rota = 0


Transicao 2
Navio1
t = 0.03343739702915857
m = HIGH
x = -0.4925770574530673
y = -0.4036250869912422
v = 10
rota = 0
Navio2
t = 0.0
m = LOW
x = 0.15100509197431786
y = -0.4901674386892491
v = 1
rota = 1
Navio3
t = 0.03343739702915857
m = HIGH
x = -0.5260144544822258
y = -1.4036250869912423
v = 10
rota = 0


Transicao 3
Navio1
t = 0.06687479405831714
m = HIGH
x = -0.20300070481629087
y = -0.23643810184544944
v = 10
rota = 0
Navio2
t = 0.03343739